# Segment_and_Cluster_Neighborhoods_in_Toronto.ipynb

This JupyterLab notebook is used for the capstone project of the course
[IBM Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone)
which is part of the overall course 
[IBM Data Science Professional Certificate](https://www.coursera.org/professional-certificates/ibm-data-science).  

## Objective of this assignment
In this assignment, it is required to explore, segment, and cluster the neighborhoods in the city of Toronto.  
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto.
It is required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.  
Once the data is in a structured format, we can explore and cluster the neighborhoods in the city of Toronto.

## Initial step
The programming language used is **Python** of the following version

In [1]:
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]


In [2]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import io

## Question 1
Obtain a DataFrame of PostalCode, Borough, and Neighborhood from [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

### Instructions (copied from assignment)
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

with requests.get(url, timeout=5) as response:
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find("table", class_="wikitable sortable")
    
    # extract header:
    header = table.tbody.find_all("tr")[0]
    heading = []
    for th in header.find_all("th"):
        # remove any newlines and strip
        heading.append(th.text.strip())
    
    # extract rows:
    rows = []
    table_rows = table.find_all('tr')
    for tr in table_rows:
        td = tr.find_all('td')
        rows.append([i.text.strip() for i in td])
    rows = rows[1:]
    
    # create DataFrame:
    df = pd.DataFrame(rows, columns=heading)
    
    # INSTRUCTION: The dataframe will consist of three columns: 
    #              PostalCode, Borough, and Neighborhood
    df.rename(columns={'Postcode': 'PostalCode',
                       'District': 'Borough', 
                       'Neighbourhood':'Neighborhood'}, inplace=True)
    
    # INSTRUCTION: Only process the cells that have an assigned borough.
    #              Ignore cells with a borough that is Not assigned:
    df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)

    # INSTRUCTION: More than one neighborhood can exist in one postal code area.
    #              Combine neighborhoods of the same borough into one row with
    #              the neighborhoods separated with a comma:
    df = df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join).reset_index()
    
    # INSTRUCTION: If a cell has a borough but a Not assigned neighborhood,
    #              then the neighborhood will be the same as the borough:
    df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
    
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [4]:
df.shape

(103, 3)

## Question 2
We need to get the latitude and the longitude coordinates of each neighborhood.  
We use a csv file that has the geographical coordinates of each postal code: [http://cocl.us/Geospatial_data](http://cocl.us/Geospatial_data)


In [5]:
url = "http://cocl.us/Geospatial_data"
with requests.get(url, timeout=5) as response:
    rc  = response.content
    nll = pd.read_csv(io.StringIO(rc.decode('utf-8')))
    nll.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
    df = df.join(nll.set_index('PostalCode'), on='PostalCode')
df.head(12)   

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


## Question 3
Explore and cluster the neighborhoods in Toronto.
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.

In [6]:
# TODO: check df for NaN / empty cells